<a href="https://colab.research.google.com/github/NaveenTark/Reinforcement-Learning-with-CPP-and-Python/blob/main/Classical_RL/ValueIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Define Environment
class GridworldMDP:
    def __init__(self, grid, start, goal, traps, obstacles, p_success=0.8, gamma=0.9):
        """
        Initialize Gridworld MDP.

        :param grid: (int, int) Grid size (rows, cols)
        :param start: (int, int) Starting position
        :param goal: (int, int) Goal position
        :param traps: List of (int, int) positions with trap rewards (-1)
        :param obstacles: List of (int, int) impassable positions
        :param p_success: Probability of moving in the intended direction
        :param gamma: Discount factor for future rewards
        """
        self.grid_size = grid

        # Validate initializations
        assert start != goal, "Start and goal positions cannot be the same."
        assert 0 <= start[0] < grid[0] and 0 <= start[1] < grid[1], "Invalid start position."
        assert 0 <= goal[0] < grid[0] and 0 <= goal[1] < grid[1], "Invalid goal position."
        assert all(0 <= trap[0] < grid[0] and 0 <= trap[1] < grid[1] for trap in traps), "Invalid trap position."
        assert all(0 <= obs[0] < grid[0] and 0 <= obs[1] < grid[1] for obs in obstacles), "Invalid obstacle position."
        assert 0 <= p_success <= 1, "Invalid success probability."
        assert 0 <= gamma <= 1, "Invalid discount factor."

        self.start = start
        self.goal = goal
        self.traps = set(traps)
        self.obstacles = set(obstacles)
        self.p_success = p_success
        self.gamma = gamma
        self.actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # UP, DOWN, LEFT, RIGHT
        self.reset()

    def reset(self):
        """Reset the agent to the start position."""
        self.agent_pos = self.start
        return self.agent_pos

    def step(self, action):
        """
        Take a step in the environment.

        :param action: (int) Action index [0=UP, 1=DOWN, 2=LEFT, 3=RIGHT]
        :return: (new_state, reward, done)
        """
        if np.random.rand() < self.p_success:
            move = self.actions[action]  # Intended move
        else:
            move = self.actions[np.random.choice(4)]  # Random move

        new_pos = (self.agent_pos[0] + move[0], self.agent_pos[1] + move[1])

        # Check for out-of-bounds or obstacle
        if (new_pos[0] < 0 or new_pos[0] >= self.grid_size[0] or
            new_pos[1] < 0 or new_pos[1] >= self.grid_size[1] or
            new_pos in self.obstacles):
            new_pos = self.agent_pos  # Stay in place

        self.agent_pos = new_pos

        # Assign rewards
        if new_pos == self.goal:
            return new_pos, 1.0, True  # Goal reached
        elif new_pos in self.traps:
            return new_pos, -1.0, True  # Trap
        else:
            return new_pos, 0.0, False  # Neutral move

    def get_transitions(self, state, action):
      """
      Return a list of transitions for a given state and action.
      Each transition is a tuple: (next_state, reward, done, probability)
      This function computes all possible outcomes given the stochastic dynamics.
      """
      outcomes = {}
      # Intended move
      intended_move = self.actions[action]
      next_state = (state[0] + intended_move[0], state[1] + intended_move[1])
      if (next_state[0] < 0 or next_state[0] >= self.grid_size[0] or
          next_state[1] < 0 or next_state[1] >= self.grid_size[1] or
          next_state in self.obstacles):
          next_state = state  # Stay in place if move is invalid
      if next_state == self.goal:
          reward, done = 1.0, True
      elif next_state in self.traps:
          reward, done = -1.0, True
      else:
          reward, done = 0.0, False

      outcomes[next_state] = (reward, done, self.p_success)

      # Random moves (with probability (1 - p_success) equally divided among all actions)
      random_prob = (1 - self.p_success) / 4.0
      for rand_action in range(4):
          rand_move = self.actions[rand_action]
          rand_state = (state[0] + rand_move[0], state[1] + rand_move[1])
          if (rand_state[0] < 0 or rand_state[0] >= self.grid_size[0] or
              rand_state[1] < 0 or rand_state[1] >= self.grid_size[1] or
              rand_state in self.obstacles):
              rand_state = state  # Stay in place if invalid
          if rand_state == self.goal:
              rand_reward, rand_done = 1.0, True
          elif rand_state in self.traps:
              rand_reward, rand_done = -1.0, True
          else:
              rand_reward, rand_done = 0.0, False

          # If outcome already exists, add probability
          if rand_state in outcomes:
              prev_reward, prev_done, prev_prob = outcomes[rand_state]
              outcomes[rand_state] = (rand_reward, rand_done, prev_prob + random_prob)
          else:
              outcomes[rand_state] = (rand_reward, rand_done, random_prob)

      transitions = []
      for ns, (rew, done, prob) in outcomes.items():
          transitions.append((ns, rew, done, prob))
      return transitions
    def get_possible_actions(self, state):
        """Returns a list of valid actions from a given state."""
        actions = []
        for i, move in enumerate(self.actions):
            new_pos = (state[0] + move[0], state[1] + move[1])
            if (0 <= new_pos[0] < self.grid_size[0] and
                0 <= new_pos[1] < self.grid_size[1] and
                new_pos not in self.obstacles):
                actions.append(i)
        return actions

    def render(self):
        """Print the grid with the agent's position."""
        grid_display = np.full(self.grid_size, ".", dtype=str)
        grid_display[self.goal] = "G"
        for trap in self.traps:
            grid_display[trap] = "T"
        for obs in self.obstacles:
            grid_display[obs] = "#"
        grid_display[self.agent_pos] = "A"  # Agent

        for row in grid_display:
            print(" ".join(row))
        print()


In [11]:
GridWorld1 = GridworldMDP((4,4), (0,0), (3,3), [(3,2)], [(1,3),(2,1)],1,0.9)

In [12]:
GridWorld1.render()

A . . .
. . . #
. # . .
. . T G



In [13]:
print(GridWorld1.agent_pos)
GridWorld1.get_transitions(GridWorld1.agent_pos, 0)

(0, 0)


[((0, 0), 0.0, False, 1.0),
 ((1, 0), 0.0, False, 0.0),
 ((0, 1), 0.0, False, 0.0)]

In [28]:
# Value iteration implementation
values = np.zeros((4, 4))
max_no_iter = 10000
threshold = 1e-6
for k in range(max_no_iter):
    copy = values.copy()
    for i in range(values.shape[0]):
        for j in range(values.shape[1]):
            state = (i, j)
            # Terminal states and obstacles handling:
            if state in GridWorld1.traps:
                values[i, j] = -1.0
                continue
            if state == GridWorld1.goal:
                values[i, j] = 1.0
                continue
            if state in GridWorld1.obstacles:
                continue

            best_action_value = -np.inf
            possible_actions = GridWorld1.get_possible_actions(state)
            for action in possible_actions:
                action_value = 0
                transitions = GridWorld1.get_transitions(state, action)
                # Sum over all transitions to compute expected value for this action
                for next_state, reward, done, prob in transitions:
                  # To avoid double counting of reward in terminal state
                  if done:
                    action_value += prob * reward
                  else:
                    action_value += prob * (reward + GridWorld1.gamma * values[next_state])
                best_action_value = max(best_action_value, action_value)
            values[i, j] = best_action_value

    # Check convergence
    if np.all(np.abs(values - copy) < threshold):
        break

print("Converged in {} iterations".format(k + 1))
print("Final state values:")
print(values)



# Policy
policy = {}
action_mapping = {0: "UP", 1: "DOWN", 2: "LEFT", 3 : "RIGHT"}  # UP, DOWN, LEFT, RIGHT
for i in range(values.shape[0]):
    for j in range(values.shape[1]):
        state = (i, j)
        if state in GridWorld1.traps:
            policy[state] = "NAN"
            continue
        if state == GridWorld1.goal:
            policy[state] = "NAN"
            continue
        if state in GridWorld1.obstacles:
            policy[state] = "NAN"
            continue
        best_action_value = -np.inf
        best_action = None
        possible_actions = GridWorld1.get_possible_actions(state)
        for action in possible_actions:
            action_value = 0
            transitions = GridWorld1.get_transitions(state, action)
            for next_state, reward, done, prob in transitions:
                if done:
                    action_value += prob * reward
                else:
                    action_value += prob * (reward + GridWorld1.gamma * values[next_state])
            if action_value > best_action_value:
                best_action_value = action_value
                best_action = action
        policy[state] = action_mapping[best_action]

print("Final policy:")
print(policy)
# Render policy as grid
print("Final policy as grid:")
grid_display = np.full(GridWorld1.grid_size, ".", dtype=str)
for state, action in policy.items():
    if action != "NAN":  # Skip terminal and obstacle states
        grid_display[state] = action
grid_display[GridWorld1.goal] = "G"  # Goal
for trap in GridWorld1.traps:
    grid_display[trap] = "T"  # Traps
for obs in GridWorld1.obstacles:
    grid_display[obs] = "#"  # Obstacles
for row in grid_display:
    print(" ".join(row))

Converged in 7 iterations
Final state values:
[[ 0.59049    0.6561     0.729      0.6561   ]
 [ 0.6561     0.729      0.81       0.       ]
 [ 0.59049    0.         0.9        1.       ]
 [ 0.531441   0.4782969 -1.         1.       ]]
Final policy:
{(0, 0): 'DOWN', (0, 1): 'DOWN', (0, 2): 'DOWN', (0, 3): 'LEFT', (1, 0): 'RIGHT', (1, 1): 'RIGHT', (1, 2): 'DOWN', (1, 3): 'NAN', (2, 0): 'UP', (2, 1): 'NAN', (2, 2): 'RIGHT', (2, 3): 'DOWN', (3, 0): 'UP', (3, 1): 'LEFT', (3, 2): 'NAN', (3, 3): 'NAN'}
Final policy as grid:
D D D L
R R D #
U # R D
U L T G


Q) Do we need to train an agent for each configuration of gridworld? Isn't that not scalable?   
Ans) PPO with randomised training can possibly help

Q) Can we train an agent to tackle multiple gridsizes with different configuration?

Q) Can we use features from agent trained on one or multiple grid size for an unknown gridsize?